In [13]:
import cv2
import os
import shutil
import json
import numpy as np

In [14]:
# Function performs SIFT on an image and returns keypoints and descriptors

def kp_des(img):
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(img, None)
    return kp, des

In [15]:
# Function for SIFT visualization returning visualized image

def viz_sift(img,kp):
    viz_img = cv2.drawKeypoints(img,kp,None,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imshow('Visualization',viz_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return viz_img

In [16]:
# Function for BF matching with KNN match (k=num_of_match) returning the match list 
# k >= 2

def match_features(section_des,whole_des,num_of_match):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(section_des,whole_des,k=num_of_match)
    return matches   

In [17]:
# Function for refining matches obtained from BF + KNN matching which returns a list of single length lists of good matches
# k >= 2
# Thus, we assume that we have for each query point at least two matches so that ratio test can be performed 
# Also, length of the returned list from match_refiner() gives us the total valid/refined/good matches
# So, separate function for total valid matches not needed

def match_refiner(matches):
    good_matches = []
    for match_list in matches:
        distance_list = []
        for individual_matches in match_list:
            distance_list.append(individual_matches.distance)
        distance_sorted = distance_list.copy()
        distance_sorted.sort()
        if (distance_sorted[0]/distance_sorted[1]) < 0.75:
            min_distance = min(distance_list)
            indx = distance_list.index(min_distance)
            best_match = match_list[indx]
            good_matches.append([best_match])
    return good_matches

# We select matches using the simple ratio test with a threshold of 0.75

In [18]:
# Function to further select top n matches from the list of good matches, returning these top matches
# input 1 = refined matches (valid, filtered matches, a query has a single match)
# input 2 = n (number of top matches needed)
# output = top matches list

def top_n_matches(good_matches,n):
    distances = []
    for match_list in good_matches:
        match = match_list[0]
        distances.append(match.distance)
    distances_np = np.array(distances)
    indx = np.argsort(distances_np)
    indx = list(indx)
    if n >= len(good_matches):
        top_matches = good_matches
    else:
        indx_slice = indx[0:n]
        top_matches = [good_matches[i] for i in indx_slice]
    return top_matches

In [19]:
# Function for match visualizer displaying the matches and returning the visualized image

def viz_match(sec,whole,kp_sec,kp_whole,matches):
    viz_img = cv2.drawMatchesKnn(sec,kp_sec,whole,kp_whole,matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv2.imshow('Visualization',viz_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return viz_img

In [20]:
# Function for the following tasks
# 1. Using valid matches to compute H via RANSAC (and returning H)
# 2. Returning relevant mask depicting inliers and outliers for visualization
# 3. Returning inliers count
# 4. Returning outliers count
# 5. Returning inlier matches
# 6. Returning top n inlier matches (with minimum error between projected source point and corresponding match point)
# 7. Returning a flag, flag = 1 if object found, flag = 0 if object not found
# Assumption - It has obviously been assumed here that top n matches refers to top n inlier matches

def compute_homography(kp_sec,kp_whole,valid_matches,match_threshold,inlier_threshold,n):
    if len(valid_matches) < match_threshold:
        flag = 0
        # print('Not enough valid matches to compute homography, object not found')
        return None,None,None,None,None,None,flag
    else:
        flag = 1
        sec_pts = np.float32([kp_sec[match[0].queryIdx].pt for match in valid_matches]).reshape(-1,1,2) 
        whole_pts = np.float32([kp_whole[match[0].trainIdx].pt for match in valid_matches]).reshape(-1,1,2)
        H,mask = cv2.findHomography(sec_pts,whole_pts,cv2.RANSAC,inlier_threshold)
        mask_list = mask.ravel().tolist()
        inlier_count = 0
        for status in mask_list:
            if status == 1:
                inlier_count = inlier_count + 1
        outlier_count = len(mask_list) - inlier_count
        inlier_matches = [valid_matches[i] for i in range(len(valid_matches)) if mask_list[i]==1]
        if n >= len(inlier_matches):
            top_inlier_matches = inlier_matches
        else:
            l2_error = []
            for match in inlier_matches:
                source_pt = np.float32(kp_sec[match[0].queryIdx].pt).reshape(-1,1,2)
                destination_pt = np.float32(kp_whole[match[0].trainIdx].pt).reshape(-1,1,2)
                projected_pt = cv2.perspectiveTransform(source_pt,H)
                l2_error.append(np.linalg.norm(destination_pt.flatten()-projected_pt.flatten()))
            l2_error_np = np.array(l2_error)
            indx = np.argsort(l2_error_np)
            indx_slice = indx[0:n]
            top_inlier_matches = [inlier_matches[i] for i in indx_slice]
        return H,mask_list,inlier_count,outlier_count,inlier_matches,top_inlier_matches,flag        

In [21]:
# Function to visualize inlier and top inlier matches

def viz_match_inlier(sec,whole,kp_sec,kp_whole,matches,flag):
    if flag == 1:
        viz_img = viz_match(sec,whole,kp_sec,kp_whole,matches)
        return viz_img
    else:
        # print('Not enough valid matches, object not found, inlier/outlier matches do not exist, refer normal matches for some visualization')
        return None   

In [22]:
# Function to visualize source object found in the destination image

def viz_found(sec,whole,kp_sec,kp_whole,matches,H,flag):
    if flag == 1:
        h,w,d = sec.shape
        edges = np.float32([[0,0],[0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)
        transformed_edges = cv2.perspectiveTransform(edges,H)
        whole_copy = whole.copy() # polylines modifies input image
        whole_new = cv2.polylines(whole_copy,[np.int32(transformed_edges)],True,255,3,cv2.LINE_AA)
        viz_img = viz_match(sec,whole_new,kp_sec,kp_whole,matches)
        return viz_img
    else:
        # print('Object not found')
        return None

In [23]:
# Reading the relevant images

tall = cv2.imread('HW3_Data\dst_0.jpg')
flat = cv2.imread('HW3_Data\dst_1.jpg')
finance = cv2.imread('HW3_Data\src_0.jpg')
deep = cv2.imread('HW3_Data\src_1.jpg')
market = cv2.imread('HW3_Data\src_2.jpg')

source = [finance,deep,market]
destination = [tall,flat]
source_names = ['finance','deep','market']
destination_names = ['tall','flat']

# The number of images are small so I have directly read them

In [24]:
source_indexer = -1
for sec in source:
    source_indexer = source_indexer + 1
    destination_indexer = -1
    for whole in destination:
        destination_indexer = destination_indexer + 1
        data_dict = {}
        data_dict['source'] = source_names[source_indexer]
        data_dict['destination'] = destination_names[destination_indexer]
        base_name = str(source_names[source_indexer]) + str('_') + str(destination_names[destination_indexer])
        if os.path.isdir(base_name):
            shutil.rmtree(base_name)
            os.mkdir(base_name)
        else:
            os.mkdir(base_name)
        kp_sec,des_sec = kp_des(sec)
        kp_whole,des_whole = kp_des(whole)
        data_dict['source_total_features'] = len(kp_sec)
        data_dict['destination_total_features'] = len(kp_whole)
        sift_sec = viz_sift(sec,kp_sec)
        sift_whole = viz_sift(whole,kp_whole)
        cv2.imwrite(str(base_name)+str('\\')+str('Source_SIFT.jpg'),sift_sec)
        cv2.imwrite(str(base_name)+str('\\')+str('Destination_SIFT.jpg'),sift_whole)
        all_matches = match_features(des_sec,des_whole,2)
        good_matches = match_refiner(all_matches)
        top_matches = top_n_matches(good_matches,20)
        data_dict['total_matches'] = len(good_matches)
        all_viz = viz_match(sec,whole,kp_sec,kp_whole,all_matches)
        good_viz = viz_match(sec,whole,kp_sec,kp_whole,good_matches)
        top_viz = viz_match(sec,whole,kp_sec,kp_whole,top_matches)
        cv2.imwrite(str(base_name)+str('\\')+str('All_Matches.jpg'),all_viz)
        cv2.imwrite(str(base_name)+str('\\')+str('Good_Matches.jpg'),good_viz)
        cv2.imwrite(str(base_name)+str('\\')+str('Top_Matches.jpg'),top_viz)
        H,mask_list,inlier_count,outlier_count,inlier_matches,top_inlier_matches,flag = compute_homography(kp_sec,kp_whole,good_matches,10,5.0,10)
        data_dict['image_found_flag'] = flag
        data_dict['homography_matrix'] = H.tolist()
        data_dict['total_inliers'] = inlier_count
        data_dict['total_outliers'] = outlier_count
        inlier_viz = viz_match_inlier(sec,whole,kp_sec,kp_whole,inlier_matches,flag)
        top_inlier_viz = viz_match_inlier(sec,whole,kp_sec,kp_whole,top_inlier_matches,flag)
        projected_viz = viz_found(sec,whole,kp_sec,kp_whole,top_inlier_matches,H,flag)
        if flag == 1:
            cv2.imwrite(str(base_name)+str('\\')+str('Inlier_Matches.jpg'),inlier_viz)
            cv2.imwrite(str(base_name)+str('\\')+str('Top_Inlier_Matches.jpg'),top_inlier_viz)
            cv2.imwrite(str(base_name)+str('\\')+str('Projected_Source.jpg'),projected_viz)
        with open(str(base_name)+str('\\')+str('Matching_Data.txt'),'w') as convert_file:
            convert_file.write(json.dumps(data_dict))   
        print(data_dict)

{'source': 'finance', 'destination': 'tall', 'source_total_features': 5259, 'destination_total_features': 49979, 'total_matches': 219, 'image_found_flag': 1, 'homography_matrix': [[0.3123722114367278, -1.1888547651125412, 2313.1550796841498], [1.322656677299396, 0.0741637350945761, -38.70257320341777], [0.0001631382402320756, -0.0002729697944438451, 0.9999999999999999]], 'total_inliers': 43, 'total_outliers': 176}
{'source': 'finance', 'destination': 'flat', 'source_total_features': 5259, 'destination_total_features': 10625, 'total_matches': 529, 'image_found_flag': 1, 'homography_matrix': [[0.3310325156783638, 0.007400795928463198, 461.44558094524683], [-0.22177282133164553, 0.29138771574706246, 202.03239040578023], [-7.14891120100088e-05, -0.00016647932449307274, 1.0]], 'total_inliers': 244, 'total_outliers': 285}
{'source': 'deep', 'destination': 'tall', 'source_total_features': 42457, 'destination_total_features': 49979, 'total_matches': 155, 'image_found_flag': 1, 'homography_matr